# Data Acquisition Example: Testing LTE-LAA/WLAN Coexistence
_Originally by Yao Ma; revised and documented by Dan Kuester_

The coexistence scenario here is the physical and MAC layers of an LTE-LAA downlink near 5 GHz (synthesized with 2 SDRs), and a COTS WLAN system is configured to operate in the same channel.

A LabView VI runs the LTE-LAA mostly as a closed-loop system. The VI includes a UDP-based socket control interface, which we use here to configure the LTE-LAA and retreive throughput information. The general implementation of the socket interface is in [labbench](https://gitlab.nist.gov/ssm/labbench), so all we need to do here is create our own specialized version, `NI_LTE_LAA_Framework`, which definees the specific commands supported by this labview VI.

Test data for WLAN is implemented with the the iperf support in [ssmdevices](https://gitlab.nist.gov/ssm/ssmdevices). It produces many more columns of data than we need, so we just use the two defined in `ipc_columns`: throughput and a timestamp.

The results of the tests and the corresponding test conditions are stored in a [flat database](https://en.wikipedia.org/wiki/Flat_file_database) stored in SQLite format. It is implemented as a typical intended use case of `labbench.RelationalDataInSQLite`. The test conditions are implemented as states in `NI_LTE_LAA_Framework` and `ssmdevices.software.IPerfClient`, so all we need to log these results into the database is to add the `db.on_set([lte_laa] + list(iperf_clients))`. Any states that are changed after that function call (for example, with the for loops in the acquisition code) become columns in the database; these values are automatically kept up to date and written to the database on calls to `db.write`.

In [ ]:
# -*- coding: utf-8 -*-
# @author: ynm5; revisions by dkuester

import time
from ssmdevices import software,instruments
import labbench as lb
import pandas as pd

class NI_LTE_LAA_Framework(lb.LabviewSocketInterface):
    resource = '127.0.0.1'
    tx_port = 61551 #  5005, LAA 
    rx_port = 61552 #  5005
    delay   = 0.1
    timeout = 27
    
    class state(lb.Device.state):
        eCCA_duration    = lb.Int(command='eCCA-duration', min=0, max=100, write_only=True)
        iCCA_duration    = lb.Int(command='iCCA-duration', min=0, max=100, write_only=True)
        eNB_power_dBm    = lb.Float(command='eNB-Power-[dBm]', min=-10, max=30, write_only=True)
        CCA_ED_threshold = lb.Float(command='CCA-ED-threshold', min=-100, max=-50, write_only=True)
        Pm               = lb.Float(command='Pm', min=0, max=1, write_only=True)
        Pf               = lb.Float(command='Pf', min=0, max=1, write_only=True)

num_trials        = 100 # number of repeat trials to run at each eCCA duration 
num_lte_laa_reads = 15 # number of LAA samples to acquire in each trial
ipc_columns       = ['iperf_bits_per_second','iperf_timestamp']
laa_columns       = 'DL throughput (current)'
db_path = 'data/{}.db'.format(time.strftime("%Y-%m-%d-%HH-%MM"))

lb.debug_to_screen(level=lb.DEBUG)

## This `with' block ensures sockets and iperf subprocesses are
## closed correctly when the script ends, even if exceptions are raised
with NI_LTE_LAA_Framework('127.0.0.1')             as lte_laa,\
     software.IPerfClient('10.1.1.20')             as iperf_laptop,\
     lb.RelationalDataInSQLite(db_path, 'results') as db:

    # add support for additional WLAN clients by instantiating them above, and
    # adding them to this list
    iperf_clients = iperf_laptop,# iperf_laptop
    iperf_laptop.debug_enable = True
    
    # Log all state changes in lte_laa and each client in iperf_clients
    db.on_set([lte_laa] + list(iperf_clients))

    # Only need to do this if there is a variable attenuator in the test setup
    # lte_enb_atten.state.attenuation = 20. # dB
    
    row = {}
    for lte_laa.state.eCCA_duration in [9,18,27,36,45]:
        print 'eCCA_duration {}'.format(lte_laa.state.eCCA_duration)

        for row['trial'] in range(num_trials):
            print '  trial {}/{}'.format(row['trial']+1,num_trials)

            # Clear old/stale data
            lte_laa.clear()            
            for ipc in iperf_clients:
                ipc.clear()
            
            # Retreive LTE results
            row['link'] = 'lte:'+lte_laa.resource
            for row['sample'] in range(num_lte_laa_reads):
                row['time'] = db.make_timestamp()
                row['throughput'] = lte_laa.read(float)[laa_columns]
                db.write(**row)
            
            # Retreive WLAN results from each iperf client
            for ipc in iperf_clients:
                row['link'] = 'wlan:'+ipc.resource
                out = ipc.fetch()[ipc_columns].values
                for row['sample'],(row['throughput'],row['time']) in enumerate(out):
                    if row['sample'] % 2: # fix dropped fraction from iperf2
                        row['time'] = pd.Timedelta(0.5, 's') + row['time']
                    db.write(**row)

# Load the sqlite database and save a copy into a csv
df = lb.read_sqlite(db_path, 'results')
df.to_csv(db_path+'.csv')